In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import featuretools as ft
import numpy as np
import pandas as pd
import os

In [ ]:
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pandas as pd
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE

from xgboost import plot_importance
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import os
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
os.chdir("/content/drive/My Drive/Competitions/AV/Janatahack Cust_Seg/")

In [ ]:
os.listdir()

['Test_LqhgPWU.csv',
 'Train_aBjfeNk.csv',
 'sample_submission_wyi0h0z.csv',
 'master_feat.csv']

In [ ]:
target = 'Segmentation'
key    = 'ID'

In [ ]:
master = pd.read_csv('master_feat.csv')
master.shape, master[key].nunique()

((10695, 112), 10695)

In [ ]:
master.set_index(key, inplace = True)

In [ ]:
catVars = ['Gender','Ever_Married','Graduated','Profession','Spending_Score','Var_1']
numVars = ['Age','Work_Experience','Family_Size']

In [ ]:
master[catVars] = master[catVars].astype(str)

### One-hot

In [ ]:
# master.dtypes

In [ ]:
categorical_features_indices = np.where((master.dtypes != 'int16') & (master.dtypes != 'int8') & (master.dtypes != 'int64') & (master.dtypes != 'float16') & (master.dtypes != 'float64'))[0]
catVars = list(set(master.iloc[:,categorical_features_indices].columns.tolist()) - set([key, target]))

In [ ]:
one_hot = pd.get_dummies(master[catVars])

master = master.drop(catVars,axis = 1)
master = master.join(one_hot)

In [ ]:
train = master[master[target] != '-1']
test  = master[master[target] == '-1']

In [ ]:
del master
import gc
gc.collect()

0

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train1, val1 = train_test_split(train, test_size=0.2, random_state=7)

In [ ]:
train1.shape, val1.shape

((6454, 150), (1614, 150))

In [ ]:
X_train = train1.drop(target, axis =1)
X_val   = val1.drop(target, axis =1)
X_test  = test.drop(target, axis =1)

y_train = train1[target]
y_val = val1[target]
y_test = test[target]

In [ ]:
###Hypertune the model intensively with a 5 fold cross validation strategy.
##first grid search for max_depth,min_child_weight then fix those and search for rest in same fashion
### parameter scale_pos_weight is quite important in case of imbalanced dataset

from sklearn.model_selection import GridSearchCV
param_test1 = {
  # 'min_child_weight':[1,50,100],
  'max_depth': [5,6,7],
  'n_estimators':[150,200,300,400],
  # 'scale_pos_weight':[1,2,3,4],
  'colsample_bytree':[0.7,0.8], 
  'subsample':[0.7,0.8]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier(tree_method='gpu_hist',predictor='gpu_predictor', learning_rate=0.1, n_estimators=1000, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=-1, scale_pos_weight=3,seed=27), 
 param_grid = param_test1, scoring=['accuracy'],n_jobs=-1,iid=False, cv=5,refit='accuracy', verbose =100)

gsearch1.fit(X_train, y_train)
# print(gsearch1.grid_scores_)
print("gsearch1.best_params_",gsearch1.best_params_)
print("gsearch1.best_score_",gsearch1.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   44.8s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


gsearch1.best_params_ {'colsample_bytree': 0.7, 'max_depth': 5, 'n_estimators': 150, 'subsample': 0.8}
gsearch1.best_score_ 0.5337747914902816


In [ ]:
%%time

modelXg = XGBClassifier(tree_method='gpu_hist',predictor='gpu_predictor',\
                        learning_rate=0.1, n_estimators=100, max_depth=5,min_child_weight=100,
                        nthread=-1, subsample=0.8, colsample_bytree=0.7, scoring = 'accuracy',seed=7)
modelXg.fit(X_train, y_train)
# scale_pos_weight=3

CPU times: user 3.07 s, sys: 540 ms, total: 3.61 s
Wall time: 1.93 s


In [ ]:
feature_important  = modelXg.get_booster().get_score(importance_type='gain')

keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score")
# data.plot(kind='barh')
data.sort_values('score', ascending = False)

,score
Spending_Score_vr.NUM_UNIQUE(trainingdata.Ever_Married),29.594649
Profession_var.SKEW(trainingdata.Family_Size),19.036536
Spending_Score_vr.MEAN(trainingdata.Age),17.577803
Spending_Score_vr.SKEW(trainingdata.Family_Size),16.201827
Spending_Score_vr.SUM(trainingdata.Age),14.765650
Spending_Score_vr.SUM(trainingdata.Work_Experience),14.553505
Spending_Score_vr.MODE(trainingdata.Ever_Married)_No,14.170802
Spending_Score_vr.STD(trainingdata.Work_Experience),13.378577
Profession_var.MODE(trainingdata.Graduated)_Yes,13.099765
Married.MEAN(trainingdata.Age),12.096695


In [ ]:
# predictions AUC on train and validation sets
print("Train Acc:", accuracy_score(y_train, modelXg.predict(X_train)))
print("Val Acc:",   accuracy_score(y_val,   modelXg.predict(X_val)))

Train Acc: 0.5788658196467307
Val Acc: 0.5322180916976456


In [ ]:
pd.DataFrame(modelXg.predict_proba(X_val)).to_csv('check.csv', index =False)

### Pred on test

In [ ]:
#### Prediction on test
output = "XGB_allFeat.csv"
test[target] = modelXg.predict(X_test)
test = test.reset_index()
# test[['Patient_ID','Health_Camp_ID']] = test[Key].str.split('|', 1, expand=True)
test[[key,target]].to_csv(output,index=False)

In [ ]:
from google.colab import files
files.download(output)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test.Crop_Damage.value_counts(1)

0    0.950481
1    0.049486
2    0.000034
Name: Crop_Damage, dtype: float64

In [ ]:
train.Crop_Damage.value_counts(1)

0    0.835468
1    0.138502
2    0.026030
Name: Crop_Damage, dtype: float64